# Bài 1

In [3]:
import pandas as pd
import sqlite3
import math

# Tạo dữ liệu
df = pd.DataFrame({
    'A': [10, 20, 30, 40, 50],
    'B': [15, 25, 35, 45, 60]
})

# Tạo database SQLite trong RAM
conn = sqlite3.connect(":memory:")
df.to_sql("data", conn, index=False, if_exists="replace")

# Truy vấn các thành phần cần thiết
query = """
SELECT 
    COUNT(*) as n,
    SUM(A) as sum_a,
    SUM(B) as sum_b,
    SUM(A*A) as sum_a2,
    SUM(B*B) as sum_b2,
    SUM(A*B) as sum_ab
FROM data;
"""

# Lấy kết quả
res = pd.read_sql_query(query, conn).iloc[0]

# Tính r_AB theo công thức
n = res['n']
sum_a = res['sum_a']
sum_b = res['sum_b']
sum_a2 = res['sum_a2']
sum_b2 = res['sum_b2']
sum_ab = res['sum_ab']

numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt(n * sum_a2 - sum_a ** 2) * math.sqrt(n * sum_b2 - sum_b ** 2)

r_ab = numerator / denominator

print("Hệ số tương quan Pearson giữa A và B là:", round(r_ab, 4))


Hệ số tương quan Pearson giữa A và B là: 0.9959


# Bài 2

In [1]:
import pandas as pd
import sqlite3

# Tạo dataframe
data = [
    ("Day 1", "A", 8),
    ("Day 1", "B", 9),
    ("Day 1", "C", 7),
    ("Day 2", "A", 7.5),
    ("Day 2", "B", 8.5),
    ("Day 2", "C", 7),
    ("Day 3", "A", 6),
    ("Day 3", "B", 7),
    ("Day 3", "C", 8),
    ("Day 4", "A", 7),
    ("Day 4", "B", 6),
    ("Day 4", "C", 5),
]

df = pd.DataFrame(data, columns=["Day", "Model", "Score"])

# Làm tròn điểm số để phù hợp với kiểm định χ²
df["RoundedScore"] = df["Score"].round()

# Kết nối SQLite
conn = sqlite3.connect(":memory:")
df.to_sql("scores", conn, index=False, if_exists="replace")
query = """
SELECT Model, RoundedScore, COUNT(*) as Frequency
FROM scores
GROUP BY Model, RoundedScore
ORDER BY Model, RoundedScore;
"""

freq_df = pd.read_sql_query(query, conn)
print(freq_df)
from scipy.stats import chi2_contingency

# Tạo bảng chéo (contingency table)
pivot = pd.pivot_table(df, values="Score", index="Model", columns="RoundedScore", aggfunc='count', fill_value=0)

# Kiểm định χ²
chi2, p, dof, expected = chi2_contingency(pivot)

print("Chi-squared =", round(chi2, 4))
print("p-value =", round(p, 4))


  Model  RoundedScore  Frequency
0     A           6.0          1
1     A           7.0          1
2     A           8.0          2
3     B           6.0          1
4     B           7.0          1
5     B           8.0          1
6     B           9.0          1
7     C           5.0          1
8     C           7.0          2
9     C           8.0          1
Chi-squared = 6.0
p-value = 0.6472


In [2]:
print('Từ kết quả ta thấy p-value = 0.6472 > 0.05 ==> không có sự khác biệt rõ ràng trong 4 ngày ')

print('và kết quả thử nghiệm phụ thuộc vào mẫu xe')

Từ kết quả ta thấy p-value = 0.6472 > 0.05 ==> không có sự khác biệt rõ ràng trong 4 ngày 
và kết quả thử nghiệm phụ thuộc vào mẫu xe


# Bài 3

In [3]:
import pandas as pd
import sqlite3

# Tạo DataFrame
data = {
    'departure_time': [830, 1445, 0, 1230, 50]
}
df = pd.DataFrame(data)

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
df.to_sql('flights', conn, index=False, if_exists='replace')

# Truy vấn SQL để chuyển đổi thời gian
query = """
SELECT 
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS departure_time_formatted
FROM 
    flights
"""
result = pd.read_sql_query(query, conn)
print(result)


   departure_time departure_time_formatted
0             830                    08:30
1            1445                    14:45
2               0                    00:00
3            1230                    12:30
4              50                    00:50


# Bài 4

In [4]:
import pandas as pd
import sqlite3
import numpy as np

# Tạo dữ liệu điểm số (Score)
data = [7, 8, 8, 7.5, 9, 10, 6, 7, 7, 8, 6, 5, 20]  # 20 là ngoại lệ

df = pd.DataFrame(data, columns=["Score"])

# Tạo kết nối SQLite và lưu vào bảng
conn = sqlite3.connect(":memory:")
df.to_sql("scores", conn, index=False, if_exists="replace")
median = df["Score"].median()
df["abs_dev"] = abs(df["Score"] - median)
mad = df["abs_dev"].median()
threshold = 1.5 * mad
# Lưu vào bảng SQLite để có thể dùng SQL lọc
df.to_sql("scores_with_dev", conn, index=False, if_exists="replace")

# Viết truy vấn SQL để lọc các điểm ngoại lệ
query = f"""
SELECT *
FROM scores_with_dev
WHERE abs(Score - {median}) > {threshold}
"""

outliers = pd.read_sql_query(query, conn)
print(" Các giá trị ngoại lệ theo MAD:")
print(outliers)


 Các giá trị ngoại lệ theo MAD:
   Score  abs_dev
0    9.0      1.5
1   10.0      2.5
2    6.0      1.5
3    6.0      1.5
4    5.0      2.5
5   20.0     12.5


MAD = Median của |x - median(X)|

Tính median(Score):
Sau khi sắp xếp: [5.0, 6.0, 6.0, 9.0, 10.0, 20.0]

Trung vị = (6.0 + 9.0) / 2 = 7.5

Tính abs_dev = |Score - median|

Tính MAD = median(abs_dev):
abs_dev = [1.5, 2.5, 1.5, 1.5, 2.5, 12.5]

Sắp xếp: [1.5, 1.5, 1.5, 2.5, 2.5, 12.5]

Median = (1.5 + 2.5)/2 = 2.0

Áp dụng ngưỡng ngoại lệ:
Ngưỡng = 1.5 * MAD = 1.5 * 2.0 = 3.0

Kiểm tra:
Chỉ có duy nhất:

|20 - 7.5| = 12.5 > 3.0
→ Score = 20.0 bị đánh dấu là ngoại lệ

# Bài 5

In [5]:
import sqlite3
import pandas as pd

# Tạo bảng dữ liệu mẫu
data = [
    ('Smith', 70, 175),
    ('Smith', 70, 180),
    ('Nguyen', 60, 160),
    ('Nguyen', 65, 165),
    ('Tran', 55, 155)
]

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Tạo bảng Patient
cur.execute('''
    CREATE TABLE Patient (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        last_name TEXT,
        weight INTEGER,
        height INTEGER
    )
''')

# Chèn dữ liệu
cur.executemany('INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)', data)
conn.commit()

# Truy vấn: So sánh cặp bệnh nhân và tính khoảng cách Boolean
query = '''
SELECT 
    a.id AS id1,
    b.id AS id2,
    a.last_name, 
    b.last_name,
    a.weight, 
    b.weight,
    CASE WHEN a.last_name = b.last_name THEN 0 ELSE 1 END AS last_name_dist,
    CASE WHEN a.weight = b.weight THEN 0 ELSE 1 END AS weight_dist,
    (CASE WHEN a.last_name = b.last_name THEN 0 ELSE 1 END +
     CASE WHEN a.weight = b.weight THEN 0 ELSE 1 END) AS total_boolean_distance,
    CASE WHEN 
        (CASE WHEN a.last_name = b.last_name THEN 0 ELSE 1 END +
         CASE WHEN a.weight = b.weight THEN 0 ELSE 1 END) = 0 
    THEN 'SAME PERSON'
    ELSE 'DIFFERENT' END AS match_result
FROM Patient a
JOIN Patient b ON a.id < b.id
'''

# Đọc kết quả
df_result = pd.read_sql_query(query, conn)
print(df_result)


   id1  id2 last_name last_name  weight  weight  last_name_dist  weight_dist  \
0    1    2     Smith     Smith      70      70               0            0   
1    1    3     Smith    Nguyen      70      60               1            1   
2    1    4     Smith    Nguyen      70      65               1            1   
3    1    5     Smith      Tran      70      55               1            1   
4    2    3     Smith    Nguyen      70      60               1            1   
5    2    4     Smith    Nguyen      70      65               1            1   
6    2    5     Smith      Tran      70      55               1            1   
7    3    4    Nguyen    Nguyen      60      65               0            1   
8    3    5    Nguyen      Tran      60      55               1            1   
9    4    5    Nguyen      Tran      65      55               1            1   

   total_boolean_distance match_result  
0                       0  SAME PERSON  
1                       2    DIFFEREN

Chỉ duy nhất cặp (id1 = 1, id2 = 2) có:

last_name = "Smith" và "Smith" → giống nhau → last_name_dist = 0

weight = 70 và 70 → giống nhau → weight_dist = 0

total_boolean_distance = 0

→ Kết luận: Chỉ có cặp (1, 2) là có thể là cùng một người.